In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import os

In [3]:
chromedriver = "/usr/bin/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [4]:
url = "https://www.tripadvisor.com/Hotels-g191-United_States-Hotels.html"

In [5]:
#set up selenium driver
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [6]:
response = requests.get(url)

In [7]:
page = response.text

In [8]:
#set up Beautiful Soup obj
soup = BeautifulSoup(page, 'lxml')

In [21]:
soup1 = BeautifulSoup(requests.get('https://www.tripadvisor.com/Hotel_Review-g60763-d8615610-Reviews-The_Beekman_A_Thompson_Hotel-New_York_City_New_York.html').text, 'lxml')

In [51]:
driver1 = webdriver.Chrome(chromedriver)
driver1.get('https://www.tripadvisor.com/Hotel_Review-g60763-d8615610-Reviews-The_Beekman_A_Thompson_Hotel-New_York_City_New_York.html')

In [11]:
soup1.find_all(attrs={'class': 'detailListItem'})

[]

In [12]:
soup1.find_all(attrs={'class':'textitem', 'data-prwidget-name': 'text'})

[<div class="textitem" data-prwidget-init="" data-prwidget-name="text">Room Service</div>,
 <div class="textitem" data-prwidget-init="" data-prwidget-name="text">Restaurant</div>,
 <div class="textitem" data-prwidget-init="" data-prwidget-name="text">Bar/Lounge</div>,
 <div class="textitem" data-prwidget-init="" data-prwidget-name="text">Free High Speed Internet (WiFi)</div>,
 <div class="textitem" data-prwidget-init="" data-prwidget-name="text">Fitness Center with Gym / Workout Room</div>,
 <div class="textitem" data-prwidget-init="" data-prwidget-name="text">Room Service</div>,
 <div class="textitem" data-prwidget-init="" data-prwidget-name="text">Wheelchair access</div>,
 <div class="textitem" data-prwidget-init="" data-prwidget-name="text">Pets Allowed ( Dog / Pet Friendly )</div>,
 <div class="textitem" data-prwidget-init="" data-prwidget-name="text">Dry Cleaning</div>,
 <div class="textitem" data-prwidget-init="" data-prwidget-name="text">Meeting Rooms</div>,
 <div class="textite

In [13]:
[entry.text.strip() for entry in soup1.find_all(attrs={'class': 'detailListItem'})]

[]

In [45]:
def getRating(driver):
    try:
        myElem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'overallRating')))
    #         return int(myElem.get_text('|').split('|')[-1][0])
    #     except TimeoutException:
    #         print('Loading took too much time!')
    #         return np.nan
    #     except:
    #         return np.nan
        return float(myElem.text.strip())
    except:
        return np.nan


In [46]:
def getRatingsoup(soup):
    try:
        return float(soup.find(attrs={'class':'overallRating'}).text[:-1])
    except:
        return np.nan

In [47]:
getRating(driver1)

5.0

In [33]:
getRatingsoup(soup1)

4.5

In [35]:
def getNumReviews(soup):
    try:
        return int(soup.find(attrs={'class':'reviewCount'}).text.split()[0].replace(',',''))
    except:
        return np.nan

In [36]:
def getCity(soup):
    try:
        return soup.find(attrs={'class':'locality'}).text.split(',')[0]
    except:
        return np.nan

In [37]:
def getState(soup):
    try:
        return soup.find(attrs={'class':'locality'}).text.split(',')[1].strip().split()[0]
    except:
        return np.nan

In [38]:
# def getStars(soup):
#     print(soup.find(attrs={'class':'prw_rup prw_common_star_rating_with_text starRatingWidget'}))
#     try:
#         return int(soup.find(attrs={'class':'prw_rup prw_common_star_rating_with_text starRatingWidget'}).get_text('|').split('|')[-1][0])
#     except:
#         return np.nan

def getStars(soup):
#     try:
#         myElem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'prw_rup.prw_common_star_rating_with_text.starRatingWidget')))
#         return int(myElem.get_text('|').split('|')[-1][0])
#     except TimeoutException:
#         print('Loading took too much time!')
#         return np.nan
#     except:
#         return np.nan
    try:
        return float(soup.find(text=re.compile(' Star accommodation')).split()[0])
    except:
        return np.nan

In [50]:
getStars(soup1)

5.0

In [40]:
def getLowestPrice(soup):
    try:
        return int(soup.find(attrs={'class':'viewAllDeals'}).text.split()[-1].strip('$.'))
    except:
#         print(soup.find(attrs={'class':'viewAllDeals'}).text)
        return np.nan

In [41]:
def getAmenitiesList(soup):
    return [entry.text.strip() for entry in soup.find_all(attrs={'class': 'detailListItem'})]

In [42]:
[x['data-index'] for x in soup.find_all(attrs={'data-url':True, 'data-index':True})]

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30']

In [48]:
#find the hotel listing tags
for entry in soup.find_all(attrs={'data-url':True, 'data-index':True}):
    #go load the page in link in each hotel
    hotel_url = 'https://www.tripadvisor.com' + entry['data-url']
#     time.sleep(.1)
    hotel_soup = BeautifulSoup(requests.get(hotel_url).text, 'lxml')
    hotel_driver = webdriver.Chrome(chromedriver).get(hotel_url) 
    print(hotel_url, getRating(hotel_driver),getRatingsoup(hotel_soup), getNumReviews(hotel_soup), getCity(hotel_soup), getState(hotel_soup),
          getStars(hotel_soup), getLowestPrice(hotel_soup))

https://www.tripadvisor.com/Hotel_Review-g60742-d12654219-Reviews-Cambria_Hotel_Suites_Downtown_Asheville-Asheville_North_Carolina.html nan 5.0 209 Asheville NC 3.5 169
https://www.tripadvisor.com/Hotel_Review-g54273-d125513-Reviews-Sonesta_Resort_Hilton_Head_Island-Hilton_Head_South_Carolina.html nan 4.5 3139 Hilton Head SC 4.0 nan
https://www.tripadvisor.com/Hotel_Review-g60814-d7216821-Reviews-Homewood_Suites_by_Hilton_Savannah_Historic_District_Riverfront-Savannah_Georgia.html nan 5.0 3116 Savannah GA 3.0 148
https://www.tripadvisor.com/Hotel_Review-g55270-d12545683-Reviews-Margaritaville_Island_Inn-Pigeon_Forge_Tennessee.html nan 4.5 309 Pigeon Forge TN 3.0 179
https://www.tripadvisor.com/Hotel_Review-g54171-d97079-Reviews-Town_Country_Inn_and_Suites-Charleston_South_Carolina.html nan 4.5 1866 Charleston SC 3.0 119
https://www.tripadvisor.com/Hotel_Review-g42671-d90199-Reviews-Best_Western_Harbour_Pointe_Lakefront-Saint_Ignace_Mackinac_County_Upper_Peninsula_Michig.html nan 4.5 18

KeyboardInterrupt: 